# Overview of Machine Learning

Machine Learning (ML) is a subfield of Artificial Intelligence that applies math and computation to build models from data. Like modeling tasks in other fields, the goal of machine learning is to cut through the messiness, chaos, and complexity of real-world data to find the underlying relationships in order to represent the system in a meaningful way.

ML algorithms use data to learn these representations, and are generally based on statistics and mathematical optimization techniques. The algorithms find patterns, trends, or learn relationships in data that may have gone unnoticed by humans.

There are different types of algorithms available, and which ones to apply will largely depend on the data and the problem at hand. Algorithms are usually grouped by the following type:

- **Supervised learning**: apply learned patterns to make accurate predictions about new, similar data
- **Unsupervised learning**: get a better understanding of the structure of the data or reduce the number of variables (or features) to what's relevant
- **Recommender systems**: make useful recommendations
- **Reinforcement learning**: find the optimal way to perform a task or learn how to interact within a system

Machine Learning is closely related to predictive statistics, and is sometimes referred to as predictive analytics or predictive modeling.

## Supervised Learning

Supervised learning is in the realm of building models to make predictions. The data that are fed to the machine learning algorithms include the "answer" or outcome the model needs to predict, which is also called the target variable or labels. The model maps the relationships between the independent variables (the input, or features) and the labels, which is later applied to new, unseen data to make predictions going forward.

Supervised learning is further broken down by the type of problem - there are **classification** problems, which predict a class or category of an observation, and **regression** problems, which predict a numerical value for an observation.

### Classification

[To Come]